## Utility Analysis of the Kompala Model
The Kompala model is an early example of the cybernetic modeling framework of [Ramkrishna and coworkers](https://engineering.purdue.edu/ramkiites/) that reimmagined the outcome of [carbon catabolite repression](https://pubmed.ncbi.nlm.nih.gov/18628769/) decisions as the output of a resource allocation problem:

* [Kompala DS, Ramkrishna D, Jansen NB, Tsao GT. (1986) Investigation of bacterial growth on mixed substrates: experimental evaluation of cybernetic models. Biotechnol Bioeng. 28:1044-55. doi: 10.1002/bit.260280715](https://pubmed.ncbi.nlm.nih.gov/18555426/)

### Background
Let's explore the ideas presented by Kompala et al. from the point of view of a resource allocation problem. In particular, let's assume the cell is trying to maximize the utility gained from consuming two possible sugars, glucose, which we denote with index `1`, and some other sugar we indicate with index `2`. Let's model the rate of growth experienced by the cell by consuming sugar $i$ as the [Monod](https://en.wikipedia.org/wiki/Monod_equation) like expression:


$$
r_{i} = \left(\frac{\mu_{i}s_{i}c}{K_{i}+s_{i}}\right)\cdot{e_{i}(R)}
$$

where $s_{i}$ denotes the concentration of sugar $i$, $\mu_{i}$ denotes the specific growth rate of the cell on sugar $i$, $K_{i}$ denotes the growth saturation constant for sugar $i$, and $c$ denotes the concentration of cells. Finally, $e_{i}(R)$ denotes the concentration of a pseudo enzyme responsible for the consumption of sugar $i$, where the expression of the pseudo enzyme $i$ depends upon some hypothetical scarce resource $R$. Assume the utility used by the cell is the linear utility function:

$$
\begin{equation}
U(e_{1},e_{2}) = \sum_{i\in{1,\dotsc,n}}r_{i}
\end{equation}
$$

where $r_{i}$ is the growth rate obtained by consuming sugar $i$. The expression of pseudo enzyme $e_{i}(R)$ is limited by the avalability of resource $R$, thus, we write a resource constraint of the form:

$$
\sum_{i\in{1,\dotsc,n}} c_{i}\cdot{e}_{i}(R) = B
$$

where $c_{i}$ denotes the resource $R$ that is consumed by the expression of $e_{i}(R)$, and $B$ denotes the total amount of resource $R$, e.g., ATP, ribosomal capacity, etc.


### Hypothesis
We hypothesize that the cell solves a resource allocation problem of the form:

$$
\begin{eqnarray}
\text{maximize}~\mathcal{O} & = & \sum_{i\in{1,\dotsc,n}}\gamma_{i}\cdot{e}_{i}(R) \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}\cdot{e}_{i}(R) & = & B \\
\text{and}~e_{i}&\geq&{0}\qquad{i=1,2,\dots,n}
\end{eqnarray}
$$

where the constants $\gamma_{i}$ is defined as:

$$
\begin{equation}
\gamma_{i} \equiv \left(\frac{\mu_{i}s_{i}c}{K_{i}+s_{i}}\right)
\end{equation}
$$

Letting the decision variable(s) be $e_{i}(R)$ gives an interesting interpretation: this is a constrained linear utility maximization problem. Thus, we expect to see corner solutions, i.e., a distinct choice between the sugars, except when the indifference curves are exactly parallel with the resource constraint.

## Setup
The computations in this lab (or example) are enabled by the [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`


## Parameters
Let's consider the case of glucose (index `1`) and xylose (index `2`):

In [23]:
s₁,s₂ = 0.5, 2.5;
K₁,K₂,μ₁,μ₂ = 0.01, 0.2, 1.08, 1.82;
β₁,β₂ = 0.05,0.05;
c = 0.35;
α₁,α₂ = 0.001, 0.001;
total_budget = 0.00010760669000000001;  # g ATP/L culture

In [24]:
γ₁ = (μ₁*s₁*c)/(K₁+s₁)

0.37058823529411766

In [25]:
γ₂ = (μ₂*s₂*c)/(K₂+s₂)

0.5898148148148147

In [26]:
c₁ = (1/(μ₁+β₁))*((α₁*s₁*c)/(K₁+s₁))

0.0003036612875238591

In [27]:
c₂ = (1/(μ₂+β₂))*((α₁*s₂*c)/(K₂+s₂))

0.00017330164388987913

## Which case should we see?

In [28]:
m_resource_line = -1*(c₁/c₂)

-1.7522123893805315

In [29]:
m_indifference = -1*(γ₁/γ₂)

-0.6283128636069815

In [30]:
ê₁ = (0.0001)/c₂

0.5770285714285717

In [31]:
if (abs(m_resource_line) < abs(m_indifference))
    println("Case A: Solution should be the x-intercept corner point")
elseif (abs(m_resource_line) > abs(m_indifference))
    println("Case B: Solution should be the y-intercept corner point")
else (abs(m_resource_line) == abs(m_indifference))
    println("Case C: We will have an infinite number of solutions")
end

Case B: Solution should be the y-intercept corner point


### Compute the optimal allocation of $e_{i}(R)$:
Analysis of the slopes of the resource budget line, and the indifference curves, suggest that the x-intercept will be the optimal solution:

$$
\hat{e}_{1} = \frac{B}{c_{1}}
$$

In [11]:
ê₁ = total_budget/c₁

0.35436420255428575

#### Solve the linear utility problem

In [32]:
# initialize -
γ = [γ₁, γ₂]; # coefficients for case A
cvector = [c₁, c₂]; # price of x1 and x2

# bounds on the choice variables -
bounds = [
    0.0 1.0; # L U
    0.0 1.0; # L U
]

# build my problem object -
problem = build(MySimpleLinearChoiceProblem, (
    
    initial = 0.1*ones(2), # initial guess
    α = γ, # coefficients
    c = cvector, # price of x1 and x2
    I = total_budget, # income
    
    # how much of x₁ and x₂ can be we buy?
    bounds = bounds
));

# call tghe solve function. This will return a dictionary -
solution = solve(problem);

In [33]:
solution

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 0.620921]
  "budget"          => 0.000107607
  "objective_value" => 0.366229